# Visualizing U.S. leading economic indicators

The following code visualizes leading economic indicators for the U.S. economy. The data is collected using custom classes (LeadingEconIndic and Idicator) and then visualized using plotly. 

*Disclaimer: This product uses the FRED® API but is not endorsed or certified by the Federal Reserve Bank of St. Louis.*

In [ ]:
# import custom module
from lei import *

In [ ]:
# import necessary libraries
import plotly.graph_objects as go
import pandas as pd
import plotly.io as pio

In [ ]:
#load indicator data
indicators = LeadingEconIndic()

In [ ]:
# this function will be used to create shaded areas for the recession indicator
def get_highliting_rectangle(start_date, end_date, fillcolor='red', opacity=0.25):
    return dict(
            type="rect",
            xref="x",
            yref="paper",
            x0=start_date,
            y0=0,
            x1=end_date,
            y1=1,
            fillcolor=fillcolor,
            opacity=opacity,
            layer="below",
            line_width=0
        )

In [ ]:
# plot all indicators
for k, v in vars(indicators).items():
    
    if k == "rec_dates":
        continue
    
    recession = indicators.rec_dates.copy()
    recession.data = recession.data[recession.data.index.intersection(v.data.index)]

    begin_dates = [recession.data.index[i] for i in range(len(recession.data)) if recession.data[i] and recession.data[i]!= recession.data[i-1]]
    end_dates =[recession.data.index[i] for i in range(len(recession.data)) if recession.data[i] and recession.data[i]!= recession.data[i+1]]

    pio.templates.default = "seaborn"
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=v.data.index, y=v.data[:],
                            line_color='dimgrey'))

    shapes = []
    for i in range(len(begin_dates)):
        shapes.append(get_highliting_rectangle(begin_dates[i],end_dates[i]))

    fig.update_layout(title_text=v.name,yaxis_title=v.unit,
        xaxis=dict(title = "Date",
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")])
            ),
        rangeslider=dict(visible=True),range = ['2007-01-01', v.data.index[-1]],type="date"), 
        shapes=shapes,
        annotations=[
            go.layout.Annotation(
            showarrow=False,
            text=v.src,
            font=dict(
            size=10,
            color="black"
            ),
            xref = 'paper', x = 0,
            yref = 'paper', y = -0.6
        ),go.layout.Annotation(
            showarrow=False,
            text="Shaded areas indicate U.S. recessions",
            font=dict(
            size=10,
            color="black"
            ),
            xref = 'paper', x = 1,
            yref = 'paper', y = 1.1
        )])

    fig.show(renderer="notebook")
